In [1]:
# -*- coding: utf-8 -*-
"""
This script runs the multi GAN and allows you to step through each part
# divide y by exposure in xpxixpy
"""

import numpy as np
# import modules
import pandas as pd
from sklearn import preprocessing

pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Import created modules

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', 999)

In [2]:
policy1 = pd.read_pickle("./data/common_dataprep/train.pickle")

cont_vars = ['VehPower', 
                     'VehAge',
                     'DrivAge',
                     'Density',
                     'BonusMalus',
            'Exposure']
cats_vars = ["ClaimNb",
            'VehBrand',
            'VehGas',
            'Region',
             'Area'
            ]

policy1[cont_vars] = policy1[cont_vars].astype(float)
policy1[cats_vars] = policy1[cats_vars].astype('category')

policy2 = pd.get_dummies(policy1[cont_vars + cats_vars])

In [3]:
ss = preprocessing.MinMaxScaler()
policy3 = pd.DataFrame(ss.fit_transform(policy2), columns = policy2.columns)

In [4]:
assert len(policy3) == len(policy2), 'Lengths shouldnt differ'

In [5]:
# Take a sampel of the data for quickly training
pol_dat  = policy3#.sample(n = 10000, random_state = 1)

second_inds = np.arange(0,(pol_dat.shape[0]-1))
val_inds = np.random.choice(second_inds, size=np.floor(pol_dat.shape[0]*.1).astype('int'), replace=False, p=None)
train_inds = np.setdiff1d(second_inds, val_inds)

val = pol_dat.iloc[val_inds]
train = pol_dat.iloc[train_inds]


In [6]:
assert policy3['Exposure'].isna().sum() == 0, 'There should not be any empty exposures'

In [7]:
train.to_pickle("./data/gan_dataprep/train_gan.pickle")
val.to_pickle("./data/gan_dataprep/val_gan.pickle")


In [8]:
import joblib
joblib.dump(ss, './data/gan_dataprep/scaler.pickle')


['./data/gan_dataprep/scaler.pickle']

In [9]:

# # Wrangle train data
# td = back_from_dummies(train_half)
# td['ClaimNb'] = td['ClaimNb'].astype('int')
# y_real, X_real = dmatrices('ClaimNb ~ CarAge + DriverAge + Power + Brand + Gas + Region + DensityCat',
#                  data=td,
#                  return_type='dataframe')
# td['Exposure'] = td['ExposureCat'].astype('float32')/11
# def xpxixpy(X,y):
#             return np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,y))
# xy = xpxixpy(X_real,y_real)


# # Fit a poisson Model
# poisson_mod = sm.GLM(y_real,X_real,family = sm.families.Poisson(), offset = td['Exposure']).fit()
# original_params = poisson_mod.params

# lower = poisson_mod.params - 1.96*poisson_mod.bse  
# upper = poisson_mod.params + 1.96*poisson_mod.bse 


# # Fit a random forest
# real_features= X_real
# real_feature_list = list(real_features.columns)
# real_features = np.array(real_features)
# y_rep = np.squeeze(y_real)/np.squeeze(td['Exposure'])
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# rf.fit(real_features, y_rep)

# # Wrangle Test Data
# test2 = back_from_dummies(test)
# test2['ClaimNb'] = test2['ClaimNb'].astype('int')
# y_test, X_test = dmatrices('ClaimNb ~ CarAge + DriverAge + Power + Brand + Gas + Region + DensityCat',
#                  data=test2,
#                  return_type='dataframe')
# test2['Exposure'] = test2['ExposureCat'].astype('float32')/11
# y_test_resp = np.squeeze(y_test)/np.squeeze(test2['Exposure'])


# # make predictions on test data with models trained on train data
# real_pois_preds = poisson_mod.predict(X_test)
# real_predictions = rf.predict(X_test)
# importances_real = rf.feature_importances_

